<a href="https://colab.research.google.com/github/aruaru0/kaggle_h_and_m/blob/main/%E5%B9%B4%E9%BD%A2%E5%88%A5%E6%9C%80%E7%B5%82%E6%9C%88%E3%81%AE%E3%83%88%E3%83%AC%E3%83%B3%E3%83%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from collections import defaultdict
import math
from tqdm import tqdm
import json

## カスタマーIDから年齢を出力するDICTを作成

In [ ]:
customers = pd.read_csv('/content/drive/MyDrive/datas/h_and_m/customers.csv',
                        usecols= ['customer_id', 'age'],
                       dtype={'customer_id': 'string'})
customers.head()

,customer_id,age
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,49.0
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,25.0
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,24.0
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,54.0
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,52.0


In [ ]:
customers['age'].min(), customers['age'].max()

(16.0, 99.0)

In [ ]:
to_age = dict()
for i in tqdm(range(len(customers))):
  customer_id, age = customers.iloc[i]
  if np.isnan(age) :
    age = 0
  to_age[customer_id] = int(age/10)

len(to_age)

100%|██████████| 1371980/1371980 [02:56<00:00, 7760.56it/s]


1371980

In [ ]:
with open("/content/drive/MyDrive/datas/h_and_m/age/cutst_age.json", "w") as f :
  json.dump(to_age, f)

## Transactionを読み込んで最終月のデータだけ抽出

In [ ]:
df  = pd.read_csv('/content/drive/MyDrive/datas/h_and_m/transactions_train.csv',
                            usecols= ['t_dat', 'customer_id', 'article_id'], 
                            dtype={'article_id': 'string', 't_dat': 'string', 'customer_id': 'string'})

df['t_dat'] = pd.to_datetime(df['t_dat'])
#last_ts = df['t_dat'].max()
df.shape

(31788324, 3)

In [ ]:
last = df[df['t_dat'] >= datetime(2020,9,1,0,0)]
last.head()

,t_dat,customer_id,article_id
30990055,2020-09-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0777148006
30990056,2020-09-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0835801001
30990057,2020-09-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0923134005
30990058,2020-09-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0865929003
30990059,2020-09-01,0005ed68483efa39644c45185550a82dd09acb07622acb...,0863646004


In [ ]:
del df

## 年齢（１０歳毎）にトレンドをデータフレームを作成

In [ ]:
# age列を追加
ages = []
for i in tqdm(range(len(last))) :
  _, customer_id, _ = last.iloc[i]
  ages.append(to_age[customer_id])

last['age'] = ages
last.head()

100%|██████████| 798269/798269 [01:45<00:00, 7563.38it/s]


,t_dat,customer_id,article_id,age
30990055,2020-09-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0777148006,4
30990056,2020-09-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0835801001,4
30990057,2020-09-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0923134005,4
30990058,2020-09-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0865929003,4
30990059,2020-09-01,0005ed68483efa39644c45185550a82dd09acb07622acb...,0863646004,7


In [ ]:
tops = [[] for _ in range(10)]
for i in range(10) :
  sel = last[last['age'] == i]
  sel.to_csv("/content/drive/MyDrive/datas/h_and_m/age/trans09_age{}.csv".format(i), index=False)
  last_sales = sel.drop(['customer_id','age'], axis=1).groupby(['article_id']).count()
  last_sales = last_sales.sort_values('t_dat', ascending=False)
  sorted = list(last_sales.index)
  tops[i] = sorted


## submission作成

In [ ]:
sub  = pd.read_csv('/content/drive/MyDrive/datas/h_and_m/sample_submission.csv',
#                            usecols= ['customer_id'], 
                            dtype={'customer_id': 'string'})

sub.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0706016001 0706016002 0372860001 0610776002 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0706016001 0706016002 0372860001 0610776002 07...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0706016001 0706016002 0372860001 0610776002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0706016001 0706016002 0372860001 0610776002 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0706016001 0706016002 0372860001 0610776002 07...


### 現状は、TOP12をそのまま

In [ ]:
import tqdm
pred = []
for i in tqdm.tqdm(range(len(sub))) :
  customer_id,  _ = sub.iloc[i]
  t = " ".join(tops[to_age[customer_id]][0:12])
  pred.append(t)
  #print(customer_id, to_age[customer_id], t)
  #sub.iloc[i]['prediction'] = t

100%|██████████| 1371980/1371980 [02:35<00:00, 8838.74it/s]


In [ ]:
sub['prediction']=pred
sub

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0751471001 0915529003 0751471043 0706016001 09...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0915526001 0909370001 0448509014 0918292001 09...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0915526001 0909370001 0448509014 0918292001 09...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0751471043 0751471001 0918522001 0924243001 08...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0751471043 0751471001 0918522001 0924243001 08...
...,...,...
1371975,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,0915526001 0909370001 0448509014 0918292001 09...
1371976,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,0915526001 0909370001 0448509014 0918292001 09...
1371977,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,0915526001 0909370001 0448509014 0918292001 09...
1371978,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,0448509014 0715624001 0685814003 0706016001 09...


In [ ]:
sub.to_csv("/content/drive/MyDrive/datas/h_and_m/age/submission.csv", index=False)

## item2vecを使ってユーザ毎に上位ｎ個から選別

In [ ]:
with open('/content/drive/MyDrive/datas/h_and_m/age/cust_buy.json', "r") as f :
  cust_buy = json.load(f)

In [ ]:
from  gensim import models

In [ ]:
model = models.Word2Vec.load('/content/drive/MyDrive/datas/word2vec.model')

In [ ]:
# model.most_similar(['0896152002'], topn = 5)

In [ ]:
# x = model.similarity(['0751471043','0924243001'], '0896152002')
# x

## item2vecを使ってユーザ毎にトレンドを並べ替え

In [ ]:
# use top n items
n = 100
pred = []

for i in tqdm.tqdm(range(len(sub))) :
  id = sub['customer_id'][i]
  age = to_age[id]
  if id not in cust_buy: 
    t = sub['prediction'][i]
    # print(t)
  else :
    buy = cust_buy[id].split(" ")
    rank = []
    for e in tops[age][0:n]:
      if e in buy :
        continue
      p =  model.similarity(buy, e).max()
      rank.append([p, e])

    rank.sort(reverse=True)
    # print()
    # print(id, age, rank[:12])
    t = " ".join([x for _, x in rank[:12]])
  pred.append(t)



100%|██████████| 1371980/1371980 [7:44:37<00:00, 49.22it/s]


In [ ]:
sub['prediction']=pred
sub.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601044 0797710001 0933706001 0911214001 08...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0781613006 0919273002 0706016001 0797988002 06...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0706016001 0610776002 0706016002 0809238001 08...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0805000001 0678942001 0751471001 0706016001 04...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0896152001 0852584001 0804992014 0918292001 08...


In [ ]:
sub.to_csv("/content/drive/MyDrive/datas/h_and_m/age/submission_item2vec.csv", index=False)